In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.layers import Dense
from tensorflow.keras import Model, layers
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
##Parameters
batch_size = 128
epochs =1

In [ ]:
##Datasets
dataset, info = tfds.load('mnist', with_info=True, as_supervised=True)
mnist_train, mnist_test = dataset['train'], dataset['test']

def convert_types(image, label):
    '''
    normarize image matrix, and reshape. (28, 29) -> (784,)
    '''
    image = tf.cast(image, tf.float32)
    image /= 255
    return tf.reshape(image,[784,]), label

mnist_train = mnist_train.map(convert_types).batch(batch_size)
mnist_test = mnist_test.map(convert_types).batch(batch_size)

In [ ]:
##Model
class Encoder(layers.Layer):
    def __init__(self):
        super(Encoder, self).__init__()
        self.d1 = Dense(units=64, activation='relu')
        self.d2 = Dense(units=64, activation='relu')
    def call(self, x):
        x = self.d1(x)
        z = self.d2(x)
        return z

class Decoder(layers.Layer):
    def __init__(self):
        super(Decoder, self).__init__()
        self.d3 = Dense(units=64, activation='relu')
        self.d4 = Dense(units=784, activation='sigmoid')
    def call(self, z):
        x = self.d3(z)
        x = self.d4(x)
        return x

class Autoencorder(Model):
    def __init__(self):
        super(Autoencorder, self).__init__()
        self.encoder = Encoder()
        self.decoder = Decoder()

    def call(self, x):
        z = self.encoder(x)
        reconstructed = self.decoder(z)
        return reconstructed

model = Autoencorder()

In [ ]:
##Setting for optimize
optimizer = tf.keras.optimizers.Adam()
train_loss = tf.keras.metrics.Mean(name='train_loss')
test_loss = tf.keras.metrics.Mean(name='test_loss')

In [ ]:
##Define train & test
@tf.function
def train_step(image):
     with tf.GradientTape() as tape:
        predictions = model(image)
        loss = tf.reduce_mean(tf.square(tf.subtract(predictions, image)))
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        train_loss(loss)
        
@tf.function
def test_step(image):
    predictions = model(image)
    t_loss = tf.reduce_mean(tf.square(tf.subtract(predictions, image)))  
    test_loss(t_loss)

In [ ]:
##Do train & test
for epoch in range(epochs):
    for image, _ in mnist_train:
        train_step(image)
  
    for test_image, _ in mnist_test:
        test_step(test_image)
  
    template = 'Epoch {}, Loss: {}, Test Loss: {}'
    print (template.format(epoch+1,
                           train_loss.result(), 
                           test_loss.result()
           )
    )

In [ ]:
##Test image
test_image = [test_image for test_image, _ in mnist_test]
plt.imshow(np.array(test_image[0][0]).reshape(28, 28))
plt.gray()

In [ ]:
##Decorded Test image
decorded_images = model(test_image[0])
decorded_images[0]
plt.imshow(np.array(decorded_images[0]).reshape(28, 28))
plt.gray()

In [ ]:
##Save model
# refered  https://www.tensorflow.org/guide/saved_model
tf.saved_model.save(model, "./")